In [ ]:
from gym import Env
from gym.spaces import Discrete, Box, Dict
import numpy as np
import random
import pandas as pd

In [ ]:
ETH_df = pd.read_csv('ETH_merged2.csv',index_col=0)
ETH_df = ETH_df.fillna(0)
ETH_df['date'] = ETH_df.index
ETH_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Open,High,Low,Close,Adj Close,Volume,daily_return,three_day_return,five_day_return,daily_up,three_day_up,five_day_up,date
2015-08-07,0.051805,-0.120037,0.001801,0.044722,0.007349,-0.032041,0.063619,0.076341,-0.079568,0.051772,0.054981,-0.039367,0.086911,0.328901,0.035358,-0.151106,-0.000778,0.012402,0.053418,0.064118,0.116689,-0.158002,0.020718,0.010733,0.035782,-0.048914,-0.147686,-0.056097,0.024108,-0.096839,0.070554,0.006699,0.015455,0.082180,0.072088,0.085314,0.026019,-0.059925,-0.278151,0.109120,...,0.157980,0.117316,0.022140,-0.035250,0.101826,-0.042502,-0.112436,-0.000079,0.046856,0.125652,-0.006029,-0.112925,0.092362,-0.089324,0.049472,-0.003427,-0.007234,0.149511,0.051480,-0.073779,-0.005037,-0.033887,0.024014,-0.102465,0.093201,-0.283361,-0.045948,2.831620,3.536610,2.521120,2.772120,2.772120,164329,0.000000,0.000000,0.000000,0,0,0,2015-08-07
2015-08-08,0.056110,0.262269,-0.006654,0.029066,0.026529,0.075086,0.059671,0.019634,0.137558,0.027349,0.039436,0.131315,0.034236,0.240227,-0.092262,-0.117125,0.029505,0.099535,0.082545,0.046965,0.031837,-0.059017,0.063357,-0.096730,-0.028039,-0.161730,0.161017,-0.031254,0.063734,-0.026130,0.112809,0.033578,-0.060393,0.037143,0.092523,0.023119,0.029368,0.057944,0.031016,0.175048,...,-0.021870,0.042102,-0.010535,0.034468,0.010670,0.013430,0.144522,0.042127,-0.006138,-0.095417,0.043582,-0.129146,0.007329,-0.002666,0.002591,-0.231551,0.021658,0.007077,-0.044012,-0.103297,-0.015089,0.021959,-0.177715,0.026050,0.104864,-0.043306,-0.010441,2.793760,2.798810,0.714725,0.753325,0.753325,674188,-0.728250,0.000000,0.000000,0,0,0,2015-08-08
2015-08-09,-0.035000,-0.046077,-0.005264,0.084920,-0.041027,-0.079960,-0.046610,-0.036378,-0.162353,0.070069,0.135490,0.073515,-0.057055,0.087793,0.062750,-0.129725,0.021179,0.051541,0.044043,0.011931,0.168629,-0.096754,0.053798,0.056459,0.044998,-0.093677,0.094365,-0.059249,0.066861,0.029302,0.011765,-0.055132,-0.024855,-0.011774,0.049611,0.051516,-0.033468,-0.006030,-0.066452,0.008814,...,-0.016306,-0.054149,-0.058821,-0.032704,-0.063529,0.093009,0.056089,0.147539,0.185581,-0.077424,0.102119,0.339049,-0.039304,-0.026432,-0.102396,0.051617,0.087242,0.129091,-0.046249,-0.099264,0.130116,-0.081820,0.139693,-0.142153,-0.033648,0.014003,0.001924,0.706136,0.879810,0.629191,0.701897,0.701897,532170,-0.068268,0.000000,0.000000,0,0,0,2015-08-09
2015-08-10,0.088439,0.069357,-0.004326,0.093559,-0.123377,-0.081841,0.157564,-0.056821,-0.144602,-0.027653,0.099649,-0.011219,-0.070312,0.184336,0.028056,-0.152006,0.016962,0.022290,-0.077808,0.032224,0.077955,-0.147059,0.019232,-0.040232,-0.020138,-0.095818,-0.107558,0.030261,-0.005725,0.186778,-0.016789,0.119854,0.016898,0.034335,0.020236,0.019849,-0.067180,0.063522,-0.147738,0.013902,...,0.016576,-0.086737,-0.039029,-0.038217,0.056407,0.117650,-0.039143,0.072820,0.033220,0.031257,-0.007120,0.070584,-0.027339,-0.038830,0.048523,0.019838,0.012785,0.135819,0.056721,0.037844,0.055416,-0.087995,0.159807,-0.109907,0.029487,-0.055866,-0.042296,0.713989,0.729854,0.636546,0.708448,0.708448,405283,0.009333,-0.744438,0.000000,1,0,0,2015-08-10
2015-08-11,-0.024601,0.118722,-0.002938,0.139157,0.042939,0.013780,-0.048534,0.060506,0.027968,0.074282,0.151977,0.082450,-0.018804,0.315835,0.009405,-0.069117,0.001729,0.005580,-0.074375,0.035962,0.039142,-0.091605,0.026238,-0.047759,0.058741,0.020346,-0.103720,-0.008060,0.138501,-0.056347,0.021653,0.049110,-0.048812,-0.007455,0.063357,-0.078218,-0.151788,-0.002016,-0.133768,-0.061940,...,0.018928,-0.127990,-0.008186,0.076947,0.034054,0.057930,0.024598,0.054505,-0.134772,-0.047699,0.001268,-0.177752,0.000528,-0.228835,0.074769,-0.138853,-0.124796,0.092865,0.038408,-0.008058,0.123892,-0.082384,0.232439,-0.092611,0.135382,-0.153356,-0.014837,0.708087,1.131410,0.663235,1.0678

In [ ]:
return_serie = ETH_df.daily_return
print(return_serie.describe())
print(return_serie.skew())
print(return_serie.kurt())
print(sum(ETH_df.daily_up))


count    2274.000000
mean        0.005382
std         0.064104
min        -0.728250
25%        -0.022798
50%         0.000854
75%         0.030349
max         0.507323
Name: daily_return, dtype: float64
0.12674384460814167
14.256068200387588
1164


In [ ]:
Lookback_period = 7

class ModelEnv(Env):
  metadata = {'render.modes': ['human']}
  def __init__ (self,data):
    super(ModelEnv, self).__init__()

    self.observation_space = Box(low = -np.inf, high = np.inf, shape = (Lookback_period,768))

    self.action_space = Discrete(2)

    self.day = len(data)
    self.day_left = self.day - Lookback_period
    self.day_curr = Lookback_period - 1
    self.data = data
    self.data_curr = self.data.loc[self.day_curr - (Lookback_period-1):self.day_curr, :]

    self.state = np.array(self.data_curr.iloc[:,0:768])

    self.reward = 0
    self.reward_hist = []
        
    
    self.done = False

  def step(self, action):

    self.day_left -= 1
    self.day_curr += 1
        
    # update stop flag
    if self.day_left == 0:
      self.done = True

    self.data_curr = self.data.loc[self.day_curr - (Lookback_period-1):self.day_curr, :]
    
    self.state = np.array(self.data_curr.iloc[:,0:768])

    price_move = np.array(self.data_curr.daily_up)[-1]

    if action == price_move:
      self.reward = 0.01
    else:
      self.reward = -0.01

    self.reward_hist.append(self.reward)

    return self.state, self.reward, self.done, {}

  def render(self):
    pass

  def reset(self):
    self.day_left = self.day - Lookback_period
    self.day_curr = Lookback_period - 1
    self.data_curr = self.data.loc[self.day_curr - (Lookback_period-1):self.day_curr, :]

    self.state = np.array(self.data_curr.iloc[:,0:768])

    self.reward = 0
    self.reward_hist = []
        
    
    self.done = False

    return self.state    

In [ ]:
start = '2015-01-01'
end = '2020-12-31'
train_data = ETH_df.loc[start:end, :]
train_data.index = train_data["date"].factorize()[0]
print("start from {}".format(train_data.loc[0, :]["date"]))
print(train_data)

start from 2015-08-07
             0         1         2  ...  three_day_up  five_day_up        date
0     0.051805 -0.120037  0.001801  ...             0            0  2015-08-07
1     0.056110  0.262269 -0.006654  ...             0            0  2015-08-08
2    -0.035000 -0.046077 -0.005264  ...             0            0  2015-08-09
3     0.088439  0.069357 -0.004326  ...             0            0  2015-08-10
4    -0.024601  0.118722 -0.002938  ...             1            0  2015-08-11
...        ...       ...       ...  ...           ...          ...         ...
1965  0.139177  0.019924 -0.007174  ...             1            1  2020-12-27
1966  0.055014 -0.090563 -0.008188  ...             1            1  2020-12-28
1967  0.057326 -0.168320 -0.002494  ...             1            1  2020-12-29
1968  0.020664  0.144040 -0.004817  ...             1            1  2020-12-30
1969  0.079555 -0.009970 -0.006170  ...             1            1  2020-12-31

[1970 rows x 781 columns]


In [ ]:
train_env = ModelEnv(train_data, 900)
episodes = 1
for episode in range(1, episodes + 1):
    state = train_env.reset()
    done = False
    score = 0
    #print(state)
    while not done:
        action = train_env.action_space.sample()
        #print(action)
        state, reward, done, info = train_env.step(action)
        #print(reward)
        #print(state)
        score += train_env.reward_hist[-1]
    print("Episode:{} Score:{}".format(episode, score))

Episode:1 Score:927


In [ ]:
!pip install stable_baselines3

In [ ]:
train_env = ModelEnv(train_data)
from stable_baselines3 import A2C
from stable_baselines3 import PPO

train_env.reset()
model1 = PPO("MlpPolicy", train_env, verbose = 1)
#model2 = DQN("MlpPolicy", train_env, verbose = 1)

model1.learn(total_timesteps = 25000)
#model2.learn(total_timesteps = 5000)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.96e+03 |
|    ep_rew_mean     | -0.87    |
| time/              |          |
|    fps             | 704      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.96e+03    |
|    ep_rew_mean          | -0.52       |
| time/                   |             |
|    fps                  | 401         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.047744144 |
|    clip_fraction        | 0.312       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [ ]:
start_test = '2021-01-01'
test_data = ETH_df.loc[start_test:, :]
test_data.index = test_data["date"].factorize()[0]
print("start from {}".format(test_data.loc[0, :]["date"]))
print(test_data)
print(sum(test_data.daily_up[5:]))

start from 2021-01-01
            0         1         2  ...  three_day_up  five_day_up        date
0   -0.020894  0.067769 -0.007911  ...             0            1  2021-01-01
1    0.106712  0.083298 -0.005509  ...             1            1  2021-01-02
2    0.058824 -0.164594 -0.004072  ...             1            1  2021-01-03
3    0.039075 -0.192720 -0.007358  ...             1            1  2021-01-04
4   -0.125209  0.227529 -0.001225  ...             1            1  2021-01-05
..        ...       ...       ...  ...           ...          ...         ...
299  0.059555 -0.006475 -0.005597  ...             0            0  2021-10-27
300  0.107976  0.159692 -0.006680  ...             1            1  2021-10-28
301  0.018310  0.182288 -0.006874  ...             1            1  2021-10-29
302  0.135713  0.120782 -0.009073  ...             1            1  2021-10-30
303 -0.015900 -0.008753 -0.002650  ...             1            1  2021-10-31

[304 rows x 781 columns]
166


In [ ]:
test_env = ModelEnv(test_data)
episodes = 10
prediction = []
for episode in range(1, episodes + 1):
    episode_prediction = []
    state = test_env.reset()
    done = False
    score = 0

    while not done:
        action, _state = model1.predict(state)
        episode_prediction.append(action)
        #print(action)
        state, reward, done, info = test_env.step(action)
        score += test_env.reward_hist[-1]
    prediction.append(episode_prediction)
    
    print("Episode:{} Score:{}".format(episode, score))

prediction_df = pd.DataFrame(prediction)
prediction_df.to_csv('PPO_pred.csv')
print(prediction_df)

Episode:1 Score:0.09
Episode:2 Score:0.07
Episode:3 Score:0.030000000000000006
Episode:4 Score:0.10999999999999999
Episode:5 Score:-0.09
Episode:6 Score:-0.05
Episode:7 Score:0.07
Episode:8 Score:0.12999999999999998
Episode:9 Score:-0.009999999999999997
Episode:10 Score:-0.009999999999999997
   0    1    2    3    4    5    6    ...  290  291  292  293  294  295  296
0    0    1    0    1    0    0    0  ...    1    0    0    1    0    0    0
1    0    1    0    1    0    0    0  ...    1    0    0    1    0    0    0
2    1    1    0    1    0    0    0  ...    1    0    0    1    0    0    0
3    0    1    0    1    0    0    0  ...    1    0    0    1    0    0    0
4    0    1    0    1    0    0    0  ...    1    0    0    1    0    0    0
5    0    1    0    1    1    0    0  ...    1    0    0    1    0    0    0
6    0    1    0    1    0    0    0  ...    1    0    0    1    0    0    0
7    0    1    0    1    1    0    0  ...    1    0    0    1    0    0    0
8    0    1   

In [ ]:
true = np.array(test_data.daily_up[7:])
pred = np.array(prediction)[2]
print(pred)

[1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1
 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1
 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0
 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 1 0
 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1
 1 1 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0 1 0 0
 0]


In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
for i in range(10):
  pred = np.array(prediction)[i]
  print('accuracy:{}'.format(accuracy_score(true, pred)))
  print('mcc:{}'.format(matthews_corrcoef(true, pred)))
  print('f1:{}'.format(f1_score(true,pred)))

accuracy:0.5151515151515151
mcc:0.04539215833092912
f1:0.5068493150684932
accuracy:0.5117845117845118
mcc:0.040821155187943416
f1:0.4982698961937716
accuracy:0.5050505050505051
mcc:0.022682426515773242
f1:0.5016949152542374
accuracy:0.5185185185185185
mcc:0.04995643734648034
f1:0.5152542372881356
accuracy:0.48484848484848486
mcc:-0.01122841879310862
f1:0.4631578947368421
accuracy:0.49158249158249157
mcc:-0.006005868329359986
f1:0.4915824915824916
accuracy:0.5117845117845118
mcc:0.03930745858305022
f1:0.5017182130584192
accuracy:0.5218855218855218
mcc:0.05906504983629012
f1:0.5136986301369862
accuracy:0.4983164983164983
mcc:0.014856965266461828
f1:0.48083623693379796
accuracy:0.4983164983164983
mcc:0.013391555434958813
f1:0.4844290657439447
